In [1]:
import pandas as pd
import numpy as np
import datetime

# O código desse notebook esta no github: https://github.com/Russel-Servicos/automacao-mkt-chat

In [2]:
caminho_do_arquivo = "/content/Tabela_SalesIQ_18_09.csv"
data = "18/09/2024"

df = pd.read_csv(caminho_do_arquivo)

In [3]:
## Remove linhas com NaN ou vazia
df.dropna(how="all")

,Visitor ID,Name,Initiated Time,Page,Mode,Question,IP,Keyword,Campaign Medium,Campaign Name,Campaign Source,State,Source
0,fc79cf92-276f-45c2-8666-66549703c3dd_a12,Visitor 901996,"18 Sep, 2024 08:00 AM",https://russelservico.com.br/servicos/recrutam...,Mobile,Quero Contratar,181.232.139.144,NaN,youtube,ygeral,google,bahia,Adwords
1,9d847865-241b-486d-902f-1a544eac4993_a12,Visitor 523874,"18 Sep, 2024 08:03 AM",https://russelservico.com.br/como-funciona,Mobile,Busco Emprego,179.247.227.195,NaN,NaN,NaN,NaN,sao paulo,Referrals
2,ac20daff-d2d4-4983-95ec-4c7d5a67ceb8_a12,Visitor 993399,"18 Sep, 2024 08:06 AM",https://russelservico.com.br/?utm_source=googl...,Mobile,Quero Contratar,170.79.199.159,NaN,youtube,ygeral,google,pernambuco,Adwords
3,f9ee2c5b-480e-4340-9dfb-41f72c6bd9f5_a12,Visitor 326633,"18 Sep, 2024 08:09 AM",https://russelservico.com.br/?utm_source=googl...,Mobile,Quero Contratar,45.176.161.170,NaN,display,remarketing,google,distrito federal,Adwords
4,b-Whatsapp_5521984850162_3040000001558043,Elane,"18 Sep, 2024 08:10 AM",NaN,NaN,Olá! Tudo bem? Gostaria de solicitar um orçame...,NaN,NaN,NaN,NaN,NaN,NaN,WhatsApp
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,b-Whatsapp_5521984850162_3040000001558584,Vanda,"18 Sep, 2024 04:48 PM",NaN,NaN,Olá! Tudo bem? Gostaria de solicitar um orçame...,NaN,NaN,NaN,NaN,NaN,NaN,WhatsApp
113,2573c9e9-a385-4740-9c86-b320ce1167b1_a12,Visitor 213451,"18 Sep, 2024 05:02 PM",https://russelservico.com.br/terceirizacao/con...,Mobile,Quero Contratar,179.68.116.95,motorista particular,pesquisa,estacionamento,google,parana,Adwords
114,d5867866-3894-497a-9729-05704f105619_a12,Visitor 323100,"18 Sep, 2024 05:55 PM",https://russelservico.com.br/,Mobile,Busco Emprego,179.126.7.181,NaN,NaN,NaN,NaN,minas gerais,Direct
115,88611ec0-f135-48e8-be6d-a1e517d8793e_a12,Visitor 336229,"18 Sep, 2024 05:58 PM",https://russelvagas.com.br/,Mobile,Quero cadastrar minha empresa.,179.126.7.181,NaN,NaN,NaN,NaN,minas gerais,Direct


In [4]:
## Remove linhas duplicadas
df = df.drop_duplicates(subset = ["Name"])

## Remove colunas
df = df.drop(columns = ['Visitor ID', 'Initiated Time', 'IP', 'Mode'], axis = 1)

In [5]:
## Muda os tipos de dados das colunas
columns_to_type_conversions={
    'Page': 'str',
    'Question':'str',
    'Source': 'str',
    'Campaign Name': 'str',
  }
df = df.astype(columns_to_type_conversions)

clear_column_nan_value = lambda column_name: np.where(df[column_name] == 'nan', '', df[column_name])

df['Page'] = clear_column_nan_value('Page')
df['Question'] = clear_column_nan_value('Question')
df['Source'] = clear_column_nan_value('Source')
df['Campaign Name'] = clear_column_nan_value('Campaign Name')

In [6]:
## Filtra as linhas a coluna Page (Página de Aterrissagem)
query_is_russelvagas = "not (Page.str.contains('russelvagas.com'))"
df = df.query(query_is_russelvagas)

In [7]:
## Renomeia colunas
columns = {
    "Name": "ID do Visitante",
    "Page": "Página de Aterrissagem",
    "Campaign Medium": "Meio",
    "Campaign Name": "Campanha",
    "Campaign Source": "Origem",
    "State": "Estado",
    "Source": "Canal"
}
df = df.rename(columns=columns)

In [8]:
# Cria novas colunas
df['Data'] = data
df['Qualificação'] = ''
df['Contagem'] = ''
df['Taxa de Conversão'] = ''

In [9]:
## Reorganiza colunas
df = df.reindex(['Data','ID do Visitante','Canal','Página de Aterrissagem','Question','Origem','Meio','Campanha','Estado','Keyword','Qualificação','Contagem','Taxa de Conversão'], axis = 1)

In [10]:
## Atribui as linhas, o valor "Emprego"
query_is_emprego = "(Question.str.contains('Emprego', case=False))"
df_is_emprego = df.query(query_is_emprego)
df_is_emprego.loc[:, "Qualificação"] = "Emprego"

query_is_not_emprego = f"not {query_is_emprego}"
df_not_is_emprego = df.query(query_is_not_emprego)

df = pd.concat([df_is_emprego, df_not_is_emprego])

In [11]:
## Deletar linhas da coluna 'Página de Aterrissagem' que conter 'russelvagas'
df = df.drop(columns = ['Question'], axis = 1)

In [12]:
# Exportar como CSV
now = datetime.datetime.now()
now = now.strftime("%Y_%m_%d_%H_%M")
df.to_csv(f'./planilha_chat_estagio_1de2_{now}.csv', index=False)